In [14]:
import spacy
from spacy.symbols import ORTH
from torchtext import data, datasets
import torch

In [15]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("Use CUDA:", USE_CUDA)

Use CUDA: True


In [7]:
bptt_len = 60
batch_size = 32

In [2]:
questions_file = 'questions.txt'

In [3]:
lm_tok = spacy.load('en')
def spacy_tok(x):
    return [tok.text for tok in lm_tok.tokenizer(x)]

TEXT = data.ReversibleField(sequential=True, tokenize=spacy_tok,
                                    lower=True, include_lengths=False)

In [4]:
questions_dataset = datasets.LanguageModelingDataset(questions_file, TEXT, newline_eos=True)

In [5]:
vectors = "glove.840B.300d"
TEXT.build_vocab(questions_dataset, vectors=vectors)

100%|██████████| 2196017/2196017 [04:27<00:00, 8221.95it/s]


In [9]:
# iterators
bpttiter = data.BPTTIterator(questions_dataset, batch_size=batch_size, bptt_len=bptt_len, repeat=False)

In [18]:
for batch in bpttiter:
    x, y = batch.text.transpose(0, 1).contiguous().to(device), \
                   batch.target.transpose(0, 1).contiguous().to(device)
    print(x.shape)
    print(y.shape)
    print(x)
    print(y)
    break

torch.Size([32, 60])
torch.Size([32, 60])
tensor([[   8,   21,    4,  ...,    7,   14,  400],
        [1891,    2,   18,  ...,   14,  711,   49],
        [   3,  338,   22,  ...,   10, 1263,    2],
        ...,
        [   3,  483,  997,  ...,    7,    6, 2134],
        [   8,   21,    4,  ...,   16,    3,   86],
        [  18,    9,   17,  ...,    9,   39,    8]], device='cuda:0')
tensor([[  21,    4,    3,  ...,   14,  400,   85],
        [   2,   18,    9,  ...,  711,   49,    2],
        [ 338,   22,    6,  ..., 1263,    2,    4],
        ...,
        [ 483,  997,    2,  ...,    6, 2134,    2],
        [  21,    4,    3,  ...,    3,   86, 2952],
        [   9,   17,    3,  ...,   39,    8,   69]], device='cuda:0')
